# Gamma/neutron discrimination based on ML

- Workflow based on R. S. Molina, I. R. Morales, M. L. Crespo, V. G. Costa, S. Carrato and G. Ramponi, "An End-to-End Workflow to Efficiently Compress and Deploy DNN Classifiers on SoC/FPGA", in IEEE Embedded Systems Letters, vol. 16, no. 3, pp. 255-258, Sept. 2024, doi: 10.1109/LES.2023.3343030.

- Code adapted from the official repository of  "An End-to-End Workflow to Efficiently Compress and Deploy DNN Classifiers on SoC/FPGA"

- Using open dataset from: https://doi.org/10.5281/zenodo.8037058

## Hardware assessment framework

---

### Import libraries

In [ ]:
#MLab UDMA library
import udma 
import matplotlib.pyplot as plt


In [ ]:
# Length of the input signal 
TRACE_LEN = 161

# Input registers
CB_INFERENCE_REG = 2 # Inference result


### Zedboard connection

In [ ]:
#IP_ADDRESS = 'XXX.XXX.XXX.XXX' # Set your ZedBoard IP Address here
IP_ADDRESS = '192.168.1.10' 
IP_PORT = 7 

In [ ]:
zedBoard = udma.UDMA_CLASS(IP_ADDRESS, IP_PORT) 

In [ ]:
connectionStatus = 0
while(connectionStatus == 0):
    connectionStatus = zedBoard.connect()

In [ ]:
zedBoard.log(0)

In [ ]:
# Function definition: used to clear out FIFO contents prior to read any value
def clearInFifo(udmaInstance):
    udmaInstance.write_reg(17, 1)
    udmaInstance.write_reg(17, 0)

### Signal to be sent to the FPGA

In [ ]:
# Gamma
# Expected value: 5
signal = [402,401,401,402,401,403,402,403,403,403,404,403,403,403,403,402,400,399,398,396,394,392,390,389,387,385,384,383,382,380,379,378,378,377,377,375,374,373,374,372,374,373,373,371,373,372,371,370,371,370,369,369,369,368,368,367,367,366,367,365,365,364,365,364,365,364,362,361,361,361,360,361,362,360,361,360,360,358,359,358,358,356,358,357,356,356,355,355,354,354,354,354,354,353,353,352,352,351,352,352,351,352,351,351,351,352,351,351,352,351,351,352,352,351,352,352,352,351,351,350,351,351,351,350,350,350,350,350,349,349,349,349,349,349,347,349,348,347,348,346,347,347,348,348,348,348,348,348,348,348,348,349,348,349,349,349,348,349,347,347,346]

# Neutron
# Expected value: 3
#signal = [404,405,404,405,404,404,405,405,405,404,405,403,404,404,404,404,402,400,397,394,392,386,381,376,371,365,359,354,349,343,337,331,324,318,312,305,298,295,286,283,276,273,268,263,259,255,249,243,240,236,232,227,225,220,218,214,211,208,204,202,198,194,191,188,185,181,179,176,173,171,167,165,163,158,158,154,152,149,146,144,141,141,137,136,132,132,129,127,125,123,121,119,119,116,114,112,111,109,108,105,104,103,102,100,98,96,95,93,92,90,89,88,87,86,84,83,81,81,79,79,76,76,75,74,72,72,71,69,69,69,69,68,69,67,68,66,66,65,65,65,64,66,64,64,64,64,64,63,63,64,62,62,61,61,60,60,59,59,59,58,59]



In [ ]:
fig = plt.figure(figsize=(13,2))
plt.plot(signal)
plt.title("Input signal")
plt.xlabel("Samples")
plt.ylabel("Amplitude (ADC units)")
plt.show()

### Interaction with ComBlock to send the signal and recive the inference result

In [ ]:
# Clear input FIFO contents to get the latest data from the FIR filter output
clearInFifo(zedBoard)

In [ ]:
# Initialize HLS IP core
zedBoard.write_reg(reg=1, data=1)

In [ ]:
# Writing to ComBlock the signal to be used as input for the inference IP core
# signal should be array-type

zedBoard.write_fifo(length=len(signal), Txdata=signal)[0] # <--- This is the signal variable that will be sent
zedBoard.write_reg(reg=2, data=1)
zedBoard.write_reg(reg=2, data=0)

### Inference result

In [ ]:
# Read inference result
zedBoard.read_reg(reg=CB_INFERENCE_REG)[1][0]

### Next step

- Use the **test.csv** dataset (located in the *dataset* folder) to generate the corresponding confusion matrix in order to evaluate the performance of the hardware implementation. 


---
